### *IPCC SR15 scenario assessment*

<img style="float: right; height: 80px; padding-left: 20px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Descriptive statistics of the primary energy development

This notebook computes indicators and diagnostics of the primary-energy timeseries by fuel
for **Table 2.6** in the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators` included in this repository.  
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r1.1.xlsx')

In [ ]:
sr1p5.load_metadata('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream, Loader=yaml.FullLoader)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
cats_15 = specs.pop('cats_15')
cats_15_no_lo = specs.pop('cats_15_no_lo')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
years = [2020, 2030, 2050]

In [ ]:
df = sr1p5.filter(category=cats_15, year=years)

## Initialize a `pyam.Statistics` instance

In [ ]:
stats = pyam.Statistics(df=df,
                        filters=[('all 1.5', {}),
                                 ('no & lo os 1.5', {'category': cats_15_no_lo}),
                                 ('hi os 1.5', {'category': ['1.5C high overshoot']})
                                ], rows=True)

In [ ]:
header='Primary energy supply (EJ)'
header_share='Share in primary energy (%)'
header_growth='Growth (factor)'

statistics_settings = dict(
    header=header,
    header_share=header_share,
    header_growth= header_growth,
    growth_year=2050,
    base_year=2020
)

In [ ]:
def add_statistics(data, base, row, growth_year, base_year,
                   header, header_share, header_growth):
    stats.add(data, header=header, row=row)
    if base is not None:
        stats.add(data / base * 100, header=header_share, row=row)
    stats.add(data[growth_year] / data[base_year] - 1,
              header=header_growth, row=row,
              subheader='{}-{}'.format(base_year, growth_year))

## Extract primary energy timeseries data and add summary statistics

In [ ]:
pe = df.filter(variable='Primary Energy').timeseries()
pe.index = pe.index.droplevel([2, 3, 4])

In [ ]:
add_statistics(pe, None, 'total primary', **statistics_settings)

## Compute share of renewables by various types in primary energy

Only use scenarios for this indicator that report both biomass and the aggregate non-biomass timeseries - otherwise, the share would be distorted.

### All renewables (biomass and non-biomass)

In [ ]:
df_pe_res = df.filter()
df_pe_res.require_variable('Primary Energy|Non-Biomass Renewables', exclude_on_fail=True)
df_pe_res.require_variable('Primary Energy|Biomass', exclude_on_fail=True)
df_pe_res.filter(exclude=False, inplace=True)

In [ ]:
res = (
    df_pe_res.filter(variable=['Primary Energy|Biomass', 'Primary Energy|Non-Biomass Renewables'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(res, pe, 'renewables', **statistics_settings)

### Biomass

In [ ]:
res_bio = (
    df.filter(variable=['Primary Energy|Biomass'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(res_bio, pe, 'biomass', **statistics_settings)

### Non-biomass renewables

In [ ]:
res_non_bio = (
    df.filter(variable=['Primary Energy|Non-Biomass Renewables'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(res_non_bio, pe, 'non-biomass', **statistics_settings)

### Renewable energy from wind and solar

As above, verify that scenarios report values for both 'Wind' and 'Solar' 

In [ ]:
df_win_sol = df.filter()
df_win_sol.require_variable('Primary Energy|Wind', exclude_on_fail=True)
df_win_sol.require_variable('Primary Energy|Solar', exclude_on_fail=True)
df_win_sol.filter(exclude=False, inplace=True)

In [ ]:
win_sol = (
    df_win_sol.filter(variable=['Primary Energy|Wind',
                                'Primary Energy|Solar'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(win_sol, pe, 'wind & solar', **statistics_settings)

## Compute share of nuclear in primary energy

In [ ]:
nuc = (
    df.filter(variable=['Primary Energy|Nuclear'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(nuc, pe, 'nuclear', **statistics_settings)

## Compute share of fossil in primary energy

In [ ]:
fossil = (
    df.filter(variable=['Primary Energy|Coal', 'Primary Energy|Gas', 'Primary Energy|Oil'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(fossil, pe, 'fossil', **statistics_settings)

In [ ]:
coal = (
    df.filter(variable=['Primary Energy|Coal'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(coal, pe, 'coal', **statistics_settings)

In [ ]:
gas = (
    df.filter(variable=['Primary Energy|Gas'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(gas, pe, 'gas', **statistics_settings)

In [ ]:
oil = (
    df.filter(variable=['Primary Energy|Oil'])
    .timeseries()
    .groupby(['model', 'scenario']).sum()
)

In [ ]:
add_statistics(oil, pe, 'oil', **statistics_settings)

## Display and export summary statistics for all 1.5C scenarios to `xlsx`

In [ ]:
summary = (
    stats.summarize(center='median', fullrange=True)
    .reindex(columns=['count', header, header_share, header_growth], level=0)
)
summary

In [ ]:
summary.to_excel('output/table_2.6_primary_energy_supply.xlsx')